<a href="https://colab.research.google.com/github/Himabindugssn/AI-pathfinding-game/blob/master/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sns

# Dataset Preparation

In [ ]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [ ]:
df=df.drop(columns=['age', 'sibsp', 'parch', 'fare','deck','embark_town','alive'])

In [ ]:
df.head()

,survived,pclass,sex,embarked,class,who,adult_male,alive,alone
0,0,3,male,S,Third,man,True,no,False
1,1,1,female,C,First,woman,False,yes,False
2,1,3,female,S,Third,woman,False,yes,True
3,1,1,female,S,First,woman,False,yes,False
4,0,3,male,S,Third,man,True,no,True


In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

In [ ]:
df=df.dropna()

In [ ]:
df['sex']=le.fit_transform(df['sex'])
df['embarked']=le.fit_transform(df['embarked'])
df['class']=le.fit_transform(df['class'])
df['who']=le.fit_transform(df['who'])
df['adult_male']=le.fit_transform(df['adult_male'])
df['alone']=le.fit_transform(df['alone'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df.head()

,survived,pclass,sex,embarked,class,who,adult_male,alive,alone
0,0,2,1,2,2,1,1,0,0
1,1,0,0,0,0,2,0,1,0
2,1,2,0,2,2,2,0,1,1
3,1,0,0,2,0,2,0,1,0
4,0,2,1,2,2,1,1,0,1


In [ ]:
df.count()

survived      889
pclass        889
sex           889
embarked      889
class         889
who           889
adult_male    889
alive         889
alone         889
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(df.drop(columns=['survived']),df['survived'],test_size=0.3)

# Naive Bayes Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class Naive_Bayes():
  def __init__(self):
      self.class_data_dic={}
  
  
      
  def fit(self,X_train,y_train):
      def generate_data(class_data_dic,X_train,y_train):
          first_one=True
          first_zero=True
          for i in range(y_train.shape[0]):
              X_temp=X_train[i,:].reshape(X_train[i,:].shape[0],1)
              if y_train[i]==1:
                  if first_one==True:
                      class_data_dic[1]=X_temp
                      first_one=False
                  else:
                      class_data_dic[1]=np.append(class_data_dic[1],X_temp,axis=1)
              elif y_train[i]==0:
                  if first_zero==True:
                      class_data_dic[0]=X_temp
                      first_zero=False
                  else:
                      class_data_dic[0]=np.append(class_data_dic[0],X_temp,axis=1)
          return class_data_dic
  
      self.X_train=X_train
      self.y_train=y_train
      self.class_data_dic[0]=np.array([[]])
      self.class_data_dic[1]=np.array([[]])
      self.class_data_dic=generate_data(self.class_data_dic,self.X_train,self.y_train)
      self.class_data_dic[0]=self.class_data_dic[0].T
      self.class_data_dic[1]=self.class_data_dic[1].T
      self.mean_0=np.mean(self.class_data_dic[0],axis=0)
      self.mean_1=np.mean(self.class_data_dic[1],axis=0)
      self.std_0=np.std(self.class_data_dic[0],axis=0)
      self.std_1=np.std(self.class_data_dic[1],axis=0)
      
      
  def predict(self,X_test):
      def posterior(X,X_train_class,mean_,std_):
          def likelyhood(x,mean,sigma):
              return np.exp(-(x-mean)**2/(2*sigma**2))*(1/(np.sqrt(2*np.pi)*sigma)) #for Gaussian Naive Bayes
          
          product=np.prod(likelyhood(X,mean_,std_),axis=1)
          product=product*(X_train_class.shape[0]/self.X_train.shape[0])
          return product
      
      p_1=posterior(X_test,self.class_data_dic[1],self.mean_1,self.std_1)
      p_0=posterior(X_test,self.class_data_dic[0],self.mean_0,self.std_0)
      return 1*(p_1>p_0)

In [ ]:
nb=Naive_Bayes()

In [ ]:
nb.fit(Xtrain.values,Ytest.values)

In [ ]:
y_pred=nb.predict(Xtest)

#Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Ytest,y_pred)

0.7265917602996255